[link text](https://)## GPU Configuration and Imports <a class="anchor" id="GPU-Configuration-and-Imports"></a>


In [23]:
import os
gpu_num = 0 # Use "" to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Colab does currently not support the latest version of ipython.
# Thus, the preview does not work in Colab. However, whenever possible we
# strongly recommend to use the scene preview mode.
try: # detect if the notebook runs in Colab
    import google.colab
    colab_compat = True # deactivate preview
except:
    colab_compat = False
resolution = [480,320] # increase for higher quality of renderings

# Allows to exit cell execution in Jupyter
class ExitCell(Exception):
    def _render_traceback_(self):
        pass

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

tf.random.set_seed(1) # Set global random seed for reproducibility


In [24]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time
import random
import matplotlib.cm as cm
from matplotlib.colors import Normalize

# Import Sionna RT components

from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Camera,Scene , antenna ,Antenna

# For link-level simulations
from sionna.channel import cir_to_ofdm_channel, subcarrier_frequencies, OFDMChannel, ApplyOFDMChannel, CIRDataset
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
from sionna.utils import compute_ber, ebnodb2no, PlotBER
from sionna.ofdm import KBestDetector, LinearDetector
from sionna.mimo import StreamManagement

In [25]:
#this functions is for writing to excel the data
import openpyxl
import os
from PIL import Image

def open_excel(file_name, sheet_name):
    """
    Open an existing Excel file or create a new one if it doesn't exist,
    and return the workbook object.
    """
    try:
        if os.path.exists(file_name):
            # Load the workbook if it exists
            workbook = openpyxl.load_workbook(file_name)
            print(f"Successfully loaded workbook: {file_name}")
        else:
            # Create a new workbook if the file does not exist
            workbook = openpyxl.Workbook()
            # Add the specified sheet name or use default 'Sheet'
            if sheet_name not in workbook.sheetnames:
                workbook.create_sheet(sheet_name)
            print(f"Created a new workbook: {file_name}")

        # Return the workbook object
        return workbook

    except Exception as e:
        print(f"Error opening or creating Excel file: {e}")
        return None

def write_to_excel(workbook, sheet_name, cell, value):
    """
    Write a value to a specific cell in the given sheet of an open workbook.
    """
    try:
        # Select the specified sheet
        if sheet_name in workbook.sheetnames:
            sheet = workbook[sheet_name]
        else:
            # If the sheet does not exist, create it
            sheet = workbook.create_sheet(sheet_name)

        # Write the value to the specified cell
        sheet[cell] = value

        print(f"Successfully wrote '{value}' to {sheet_name} at {cell}.")

    except Exception as e:
        print(f"Error writing to Excel file: {e}")

def save_excel(workbook, file_name):
    """
    Save the workbook to the specified file.
    """
    try:
        workbook.save(file_name)
        print(f"Workbook saved to {file_name}")
    except Exception as e:
        print(f"Error saving Excel file: {e}")

# Example usage:
file_name = "map_to_images.xlsx"
sheet_name = "Sheet"

# Open the Excel file (or create it if it doesn't exist)
workbook = open_excel(file_name, sheet_name)

Successfully loaded workbook: map_to_images.xlsx


Loading scene

In [ ]:
# Load integrated scene
#scene = load_scene(sionna.rt.scene.munich) # Try also sionna.rt.scene.etoile
#scene = load_scene("/content/test_terrain.xml")
#scene = sionna.rt.load_scene(filename="/content/drive/MyDrive/cities/munich_1/munich.xml")
scene = sionna.rt.load_scene(filename="/content/drive/MyDrive/cities/nasr_city/Nasr_city.xml")

In [ ]:
#testing here the max size of the map
#alright to get the max postion of a point do the followin
#get the center and the size using scene.center and scene.size
#max and min is center + or - size /2
print(scene.size)
tx = Transmitter(name="tx",
                 position=[-67.850464, -85.782776 ,  49.285])#[8.5,1000,27])
#scene.remove("tx")
scene.add(tx)
tx.position = [-67.850464,-85.782776,49.825]
tx.position = scene.center
#tx_or = tx.orientation
print(tx.orientation)
#+ 1205.6357/2

In [ ]:
!mkdir images
#os.makedirs('./images/base_map', exist_ok=True)
# %cd images
# %mkdir

In [ ]:
1381/np.sin(np.deg2rad(45))

In [ ]:
max(scene.size.numpy()[0:1])

In [ ]:
tx_pos = scene.transmitters["tx"].position.numpy()
tx_or = scene.transmitters["tx"].orientation.numpy()
bird_pos = tx_pos.copy()
bird_pos = scene.center.numpy()
brid_pos_h = max(scene.size.numpy())/np.sin(np.deg2rad(45))
print(brid_pos_h)
#bird_pos[-1] = 1781 Set height of coverage map to 1000m above tx
bird_pos[-1] = brid_pos_h
bird_pos[-2]-= 0.21 # Slibrid_pos_hghtly move the camera for correct orientation
#bird_cam = Camera("birds_view", position=bird_pos, orientation = [1.57,1.57,-0.001])
bird_cam = Camera("birds_view", position=bird_pos, look_at=tx_pos)
#bird_cam = Camera("birds_view", position=bird_pos, look_at=tx_pos)
#print(bird_cam.orientation)
#scene.remove("birds_view")
try:
  scene.add(bird_cam)
except:
  scene.remove("birds_view")
  scene.add(bird_cam)
base_map_image_path = './images/base_map.png'
scene.render_to_file(camera=bird_cam,filename=base_map_image_path,resolution=(655, 500))


In [ ]:
# !zip -r /content/images_gen.zip /content/images
# from google.colab import files
# files.download("/content/images_gen.zip")

In [ ]:
#here defining the excel file header
#save this to a file
excel_file_name = "map_to_images.xlsx"
sheet_name = "Sheet"

# Open the Excel file (or create it if it doesn't exist)
workbook = open_excel(file_name, sheet_name)

#header for the excel file
write_to_excel(workbook, sheet_name, "A1", "base_map")  #possible
write_to_excel(workbook, sheet_name, "B1", "frequency")   #possible
write_to_excel(workbook, sheet_name, "C1", "terrain")   #exists but in blender
write_to_excel(workbook, sheet_name, "D1", "azimuth_angles") #possible
write_to_excel(workbook, sheet_name, "E1", "transmitter_tilt")    #in progress
#write_to_excel(workbook, sheet_name, "D1", "antenna_pattern_vertical") #possbile deprecated
#write_to_excel(workbook, sheet_name, "E1", "antenna_pattern_horizontal")  #possible deprecated
write_to_excel(workbook, sheet_name, "F1", "transmitter_locations")   #possible
write_to_excel(workbook, sheet_name, "G1", "transmitter_height")    #done in a different way
write_to_excel(workbook, sheet_name, "H1", "antenna_pattern_vetical") #possible
write_to_excel(workbook, sheet_name, "I1", "antenna_pattern_horizontal") #possible

#write_to_excel(workbook, sheet_name, "I1", "gis_data")    #not yet future

write_to_excel(workbook, sheet_name, "J1", "coverage_map")


save_excel(workbook, file_name)
terrain_image_path = os.path.join('./images', 'terrain_1.png')

In [ ]:
#defineing the transmitters here
tx1 = Transmitter(name="tx1",position=[0,0,0])
tx2 = Transmitter(name="tx2",position=[0,0,0])
tx3 = Transmitter(name="tx3",position=[0,0,0])
tx4 = Transmitter(name="tx4",position=[0,0,0])
tx5 = Transmitter(name="tx5",position=[0,0,0])
trans_list = [tx1,tx2,tx3,tx4,tx5]


# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="tr38901",
                             polarization="V")


scene.rx_array = PlanarArray(num_rows=1,
                          num_cols=1,
                          vertical_spacing=0.5,
                          horizontal_spacing=0.5,
                          pattern="dipole",
                          polarization="cross")


In [ ]:
from matplotlib import cm
#bird_cam.position
#scene.tx_array.antenna._patterns[0]()
#testant =Antenna("tr38901", "VH")
#testant.patterns[0]
#antenna.visualize(testant.patterns[0])
#print(scene.tx_array.antenna.compute_gain())
#methods_list = [method for method in dir(antenna) if callable(
#    getattr(antenna, method)) and not method.startswith("__")]
#print(methods_list)
#print(dir(scene.tx_array.antenna))


#2d attern horizontal cut
# PI= np.pi
# phi = np.linspace(-PI, PI, 1000)
# c_theta, c_phi = scene.tx_array.antenna._patterns[0]((PI/2*tf.ones_like(phi)) , tf.constant(phi, tf.float32))
# print(c_theta)
#c_theta, c_phi = scene.tx_array.antenna._patterns[0]()


pattern_image_path = 'images'
# 3D visualization
def save_antenna_pattern_img(scene,save_path,id):
  plt.ioff()
  pattern = scene.tx_array.antenna._patterns[0]
  PI= np.pi
  theta = np.linspace(0.0, PI, 50)
  phi = np.linspace(-PI, PI, 50)
  theta_grid, phi_grid = np.meshgrid(theta, phi, indexing='ij')
  c_theta, c_phi = pattern(theta_grid, phi_grid)
  g = np.abs(c_theta)**2 + np.abs(c_phi)**2
  x = g * np.sin(theta_grid) * np.cos(phi_grid)
  y = g * np.sin(theta_grid) * np.sin(phi_grid)
  z = g * np.cos(theta_grid)

  g = np.maximum(g, 1e-5)
  g_db = 10*np.log10(g)

  def norm(x, x_max, x_min):
      """Maps input to [0,1] range"""
      x = 10**(x/10)
      x_max = 10**(x_max/10)
      x_min = 10**(x_min/10)
      if x_min==x_max:
          x = np.ones_like(x)
      else:
          x -= x_min
          x /= np.abs(x_max-x_min)
      return x

  g_db_min = np.min(g_db)
  g_db_max = np.max(g_db)

  fig_3d = plt.figure()
  #plt.grid(False)
  ax = fig_3d.add_subplot(1,1,1, projection='3d')
  #ax.get_xaxis().set_visible(False)
  #ax.get_yaxis().set_visible(False)
  plt.axis('off')
  ax.plot_surface(x, y, z, rstride=1, cstride=1, linewidth=0,
                  antialiased=False, alpha=0.7,
                  facecolors=cm.viridis(norm(g_db, g_db_max, g_db_min)))

  image_path = os.path.join(save_path, f'antenna_pattern_{id}.png')

  # Save the image to the specified path
  plt.savefig(image_path)
  print(f"Image saved to {image_path}")
  return image_path

#uncomment those later
#im1,im2,im3 = antenna.visualize(scene.tx_array.antenna._patterns[0])
#vertical_pattern_path = './images/vertical_pattern.png'
#horizontal_pattern_path = './images/horizontal_pattern.png'


In [ ]:
#save_antenna_pattern(scene,'./images',1)

In [ ]:
#im3.axes.remove([str(x) for x in im3.axes])

In [ ]:
def remove_all_transmitters(scene):
  for x in scene.transmitters.keys():
    scene.remove(x)
  return


def number_to_viridis_image(number, save_path,id,min_val=1.5, max_val=3000):
    # Ensure the number is within the provided range
    if number < min_val or number > max_val:
        raise ValueError(f"Number must be between {min_val} and {max_val}")

    # Normalize the input number to a value between 0 and 1
    norm = Normalize(vmin=min_val, vmax=max_val)
    normalized_number = norm(number)

    # Get the corresponding Viridis color
    viridis_color = cm.viridis(normalized_number)

    # Create a small image (e.g., 100x100 pixels) with the color
    image = np.ones((655,500, 3)) * viridis_color[:3]  # Ignore alpha channel

    # Display the image
    #plt.imshow(image)
    #plt.axis('off')  # Hide axes
    image_path = os.path.join(save_path, f'antenna_height{id}.png')
    plt.imsave(image_path, image)

    return image_path

def number_to_greyscale_image(Value, save_path, id, Variable_Name, min_val=1.5, max_val=3000):
    # Ensure the Value is within the provided range
    if Value < min_val or Value > max_val:
        raise ValueError(f"Number must be between {min_val} and {max_val}")

    # Normalize the input number to a value between 0 and 1
    norm = Normalize(vmin=min_val, vmax=max_val)
    normalized_number = norm(Value)

    # Get the corresponding greyscale value (normalize from 0 to 1, then map to greyscale)
    greyscale_value = normalized_number * np.ones(3)  # Same value for R, G, B

    # Create a small image (e.g., 655x500 pixels) with the greyscale color
    image = np.ones((655, 500, 3)) * greyscale_value  # Each pixel gets the same grey value

    # Ensure the directory exists
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # Save the image
    image_path = os.path.join(save_path, f'{Variable_Name}_{id}.png')
    plt.imsave(image_path, image)

    return image_path


def plot_antenna_heights(heights, positions,save_path,id,min_width, max_width,image_width = 500,image_height= 600,min_val=1.5, max_val=3000):
  #image = np.zeros(image_width, image_height, 3)  # Initialize with black (RGB = 0,0,0)

  norm = Normalize(vmin=min_val, vmax=max_val)
  normalized_heights = norm(heights)

  heights_color = cm.viridis(normalized_heights)


  img = Image.new('RGB', (image_width, image_height), 'black')
  counter = 0
  for pos in positions:
    normalized_x = (pos[0] - min_width) / (max_width - min_width)
    normalized_y = (pos[1] - min_width) / (max_width - min_width)
    # Determine the pixel position
    pixel_x = int(normalized_x * (image_width - 1))
    pixel_y = int(normalized_y * (image_height - 1))
    img.putpixel((pixel_x, pixel_y), heights_color[counter])
    counter = counter + 1

  end_part = "trans_heights" + str(id) + '.png'
  image_path = os.path.join(save_path , end_part)
  print(image_path)

  img.save(image_path)
  return image_path

def calculate_tilt(img_h,img_w,antenna_pos, antenna_direction):
  #here i am trying to walk throught the tilt code step by step
  y,x = np.indices((img_w,img_h))
  pixel_coords = np.stack((x, y), axis=-1)
  z_element = np.zeros((img_w, img_h, 1), dtype=pixel_coords.dtype)
  pixel_coords = np.concatenate((pixel_coords, z_element), axis=-1)

  pointing_vec = np.array([-1,-1,0]) #it is y first and then inverted x
  pointing_vec = pointing_vec/np.linalg.norm(pointing_vec)
  #ant_position = [159,150,15]
  the_vals_arr = pixel_coords - antenna_pos

  mags_arr = np.linalg.norm(the_vals_arr, axis=-1, keepdims=True)
  mags_arr = np.where(mags_arr == 0, 1e-10, mags_arr)
  normailize_arr = the_vals_arr / mags_arr

  new_normalize = np.zeros((img_w,img_h, 2), dtype=float)
  new_normalize[:, :, 0] = np.sqrt(normailize_arr[:, :, 0]**2 + normailize_arr[:, :, 1]**2)  # Sum of first and second elements
  #new_normalize[:, :, 0] = normailize_arr[:, :, 1]
  new_normalize[:, :, 1] = normailize_arr[:, :, 2]


  # #final_image_vals = np.arctan2(normailize_arr[:,:,1],normailize_arr[:,:,0])
  # before_dot = np.dot(normailize_arr , pointing_vec)#(np.linalg.norm(normailize_arr, axis=-1) * np.linalg.norm(pointing_vec))
  # mags_dot = np.linalg.norm(before_dot, axis=-1, keepdims=True)
  # before_dot = before_dot/mags_dot
  # #print(before_dot[0])
  # before_dot = np.arccos(before_dot)
  # #trying the dor product method
  # new_dotter = np.zeros((300, 300), dtype=float)
  # z_dir = np.array([0,0,1])
  # # for i , big_arr in enumerate(normailize_arr):
  # #   for j,temp_vec in enumerate(big_arr):
  # #     #print(temp_vec)
  # #     new_dotter[i,j] = angle_between(temp_vec,z_dir)
  # # ant_ver_angle = angle_between(pointing_vec,z_dir)
  # # new_dotter = new_dotter - ant_ver_angle

  angle_diff_1 = np.arctan2(new_normalize[:,:,1],new_normalize[:,:,0])
  angle_diff_2 = np.arctan2(np.sqrt(pointing_vec[1]**2+pointing_vec[0]**2),pointing_vec[2])
  #print(np.degrees(angle_diff_2))
  final_diff = angle_diff_1 - angle_diff_2
  final_diff = np.where(final_diff >= -3.14,final_diff , np.pi + (final_diff+3.14))
  final_diff = np.where(final_diff <= 3.14,final_diff , np.pi - (final_diff-3.14))
  final_image_vals = np.degrees(final_diff)
  return final_image_vals

In [ ]:
from PIL import Image, ImageDraw
import math
#non loop imagees that needs to be saved assuming that there is only one map for now
#or to be more specfic if there is will be a loop on many maps this only happens once per map
#maybe if we will change the antenna array then stuff would be different
def save_antenna_pattern(fig_name):
  im1,im2,im3 = antenna.visualize(scene.tx_array.antenna._patterns[0])
  im1.savefig(fig_name + 'V')
  im2.savefig(fig_name + 'H')

#this funciton needs more thniking
def create_colored_image_from_number(input_number, save_path):
    """
    Create an image of a specific color based on a large input number.

    Parameters:
    input_number (int): A large integer number that determines the color.
    save_path (str): Path to save the generated image.
    """

    # Example transformation: use modulus with different prime numbers to distribute colors
    input_number = int(input_number / 1000)
    red = input_number % 256
    green = (input_number // 256) % 256
    blue = (input_number // (256 * 256)) % 256

    # The color is determined by the mapped RGB values
    color = (red, blue , green)

    # Define image size (e.g., 100x100 pixels)
    img_size = (655,500)

    # Create an image with the specified color
    img = Image.new('RGB', img_size, color)

    # Define the output file path
    image_path = os.path.join(save_path, f'colored_image_{input_number}.png')

    # Save the image to the specified path
    img.save(image_path)
    #print(f"Image saved to {image_path}")
    return image_path


def save_antenna_pos(positions, min_width, max_width, min_height, max_height,
                     image_size, save_path,img_id):

  """
  this function saves the postion of antennas and saves them on a black image as white dots
  """
  img_width,img_height = image_size
  img = Image.new('RGB', (img_width, img_height), 'black')
  for pos in positions:
    normalized_x = (pos[0] - min_width) / (max_width - min_width)
    normalized_y = (pos[1] - min_width) / (max_width - min_width)
    # Determine the pixel position
    pixel_x = int(normalized_x * (img_width - 1))
    pixel_y = int(normalized_y * (img_height - 1))
    img.putpixel((pixel_x, pixel_y), (255, 255, 255))
  end_part = "trans_loc" + str(img_id) + '.png'
  image_path = os.path.join(save_path , end_part)
  print(image_path)

  img.save(image_path)
  return image_path


import numpy as np

def euler_to_cartesian(yaw, pitch, roll):
    # Convert angles from degrees to radians
    #yaw = np.radians(yaw)
    #pitch = np.radians(pitch)
    #roll = np.radians(roll)

    # Define the rotation matrices
    R_z = np.array([
        [np.cos(yaw), -np.sin(yaw), 0],
        [np.sin(yaw), np.cos(yaw), 0],
        [0, 0, 1]
    ])

    R_y = np.array([
        [np.cos(pitch), 0, np.sin(pitch)],
        [0, 1, 0],
        [-np.sin(pitch), 0, np.cos(pitch)]
    ])

    R_x = np.array([
        [1, 0, 0],
        [0, np.cos(roll), -np.sin(roll)],
        [0, np.sin(roll), np.cos(roll)]
    ])

    # Compute the combined rotation matrix
    R = R_z @ R_y @ R_x

    # Define the initial vector (unit vector along x-axis)
    V = np.array([1, 0, 0])

    # Apply the rotation
    V_prime = R @ V

    return V_prime

# # Example usage
# yaw = 30  # in degrees
# pitch = 45 # in degrees
# roll = 60  # in degrees

# cartesian_coordinates = euler_to_cartesian(yaw, pitch, roll)
# print(f"Cartesian Coordinates: {cartesian_coordinates}")


def save_antenna_orientation(positions,orientation ,min_width, max_width, min_height, max_height,
                     image_size, save_path,img_id):

  """
  this function saves the postion of antennas and saves them on a black image as white dots
  expected orientation is in rad not in degrees
  """
  img_width,img_height = image_size
  img = Image.new('RGB', (img_width, img_height), 'black')
  draw = ImageDraw.Draw(img)

  counter = 0
  for pos in positions:
    # print(pos)
    normalized_x = (pos[0] - min_width) / (max_width - min_width)
    normalized_y = (pos[1] - min_width) / (max_width - min_width)
    # Determine the pixel position
    pixel_x = int(normalized_x * (img_width - 1))
    pixel_y = int(normalized_y * (img_height - 1))
    img.putpixel((pixel_x, pixel_y), (255, 255, 255))

    result_rad = euler_to_cartesian(orientation[counter][0],orientation[counter][1],orientation[counter][2])
    x_angle_rad = result_rad[0]
    y_angle_rad = result_rad[1]
    #z_angle_rad = result_rad[2]
    z_angle_rad = result_rad[0]
    print(result_rad)

    vector_length = 20
    end_x = pixel_x + vector_length * math.cos(z_angle_rad)
    end_y = pixel_y - vector_length * math.sin(z_angle_rad)  # Subtract to invert y-axis

    # Draw the main line of the vector
    draw.line((pixel_x, pixel_y, end_x, end_y), fill='white', width=2)

    # Draw the arrowhead
    arrow_size = 5
    left_arrow_angle = z_angle_rad + math.radians(150)  # Left side of the arrowhead
    right_arrow_angle = z_angle_rad - math.radians(150)  # Right side of the arrowhead

    left_arrow_x = end_x + arrow_size * math.cos(left_arrow_angle)
    left_arrow_y = end_y - arrow_size * math.sin(left_arrow_angle)

    right_arrow_x = end_x + arrow_size * math.cos(right_arrow_angle)
    right_arrow_y = end_y - arrow_size * math.sin(right_arrow_angle)

    # Draw the arrowhead lines
    draw.line((end_x, end_y, left_arrow_x, left_arrow_y), fill='white', width=2)
    draw.line((end_x, end_y, right_arrow_x, right_arrow_y), fill='white', width=2)
    counter = counter + 1

  end_part = "trans_ori" + str(img_id) + '.png'
  image_path = os.path.join(save_path , end_part)
  print(image_path)

  img.save(image_path)
  return image_path



def calculate_azimuth_angles(image_height,image_width, antenna_pos, antenna_direction):
  """
  Calculate the azimuth angle of each pixel relative to the antenna's main lobe direction.

  Parameters:
      pixel_coords (numpy array): Array of shape (H, W, 2) containing the coordinates of each pixel.
      antenna_pos (tuple): (x0, y0) Position of the antenna in the image.
      main_lobe_azimuth (float): Azimuth angle of the antenna's main lobe in radians.

  Returns:
      numpy array: Array of azimuth angles in degrees for each pixel.
  """
  y, x = np.indices((image_height,image_width))
  pixel_coords = np.stack((x, y), axis=-1)
  antenna_direction = euler_to_cartesian(antenna_direction[0],antenna_direction[1],0)
  #the_3edla = pixel_coords
  #pointing_vec = np.array([0,1]) #it is y first and then inverted x
  antenna_direction = antenna_direction/np.linalg.norm(antenna_direction)
  #ant_position = [150,150]
  LOS_matrix = pixel_coords - antenna_pos
  #print(the_3edla)
  #print('--------')
  #print(the_vals_arr)
  #the_3edla[0,0] = 1e-10
  pixel_magnitudes = np.linalg.norm(LOS_matrix, axis=-1, keepdims=True)
  pixel_magnitudes = np.where(pixel_magnitudes == 0, 1e-10, pixel_magnitudes)
  pixel_normalized = LOS_matrix / pixel_magnitudes
  #print(normailize_arr)
  #final_image_vals = np.arctan2(normailize_arr[:,:,1],normailize_arr[:,:,0])
  #before_dot = np.dot(normailize_arr , pointing_vec)
  pixel_horizontal = np.arctan2(pixel_normalized[:,:,1],pixel_normalized[:,:,0])
  antenna_horizontal = np.arctan2(antenna_direction[1],antenna_direction[0])
  azimuth_angles = pixel_horizontal - antenna_horizontal
  azimuth_angles = np.where(azimuth_angles >= -3.14,azimuth_angles , np.pi + (azimuth_angles+3.14))
  azimuth_angles = np.where(azimuth_angles <= 3.14,azimuth_angles , np.pi - (azimuth_angles-3.14))
  azimuth_angles = np.degrees(azimuth_angles)

  return azimuth_angles


def plot_azimuth(arr_azimuth_angles,save_path,id,tilt_azim):
  azimuth_angles = arr_azimuth_angles[0]
  for i in range(1,len(arr_azimuth_angles)):
    azimuth_angles = azimuth_angles + arr_azimuth_angles[i]

  print(azimuth_angles.shape)
  normalized_azimuth = (azimuth_angles - azimuth_angles.min()) / (azimuth_angles.max() - azimuth_angles.min()) #this may need to change
  # Get the colormap function
  colormap_func = cm.get_cmap('viridis')

  # Apply the colormap to the normalized azimuth angles
  color_image = colormap_func(normalized_azimuth)[:, :, :3]  # Use only RGB channels, ignore alpha

  #figure()
  plt.ioff()
  color_image = (color_image * 255).astype(np.uint8)
  plt.imshow(color_image,cmap='gray')

  if tilt_azim == 1:
    image_path = os.path.join(save_path, f'azimuth_{id}.png')
  else:
    image_path = os.path.join(save_path, f'tilt_{id}.png')


  # Save the image to the specified path
  plt.savefig(image_path)
  print(f"Image saved to {image_path}")
  return image_path


def sionna_to_image_cord(pos,min_width, max_width, min_height, max_height,image_size):
  img_width,img_height = image_size
  normalized_x = (pos[0] - min_width) / (max_width - min_width)
  normalized_y = (pos[1] - min_width) / (max_width - min_width)
  pixel_x = int(normalized_x * (img_width - 1))
  pixel_y = int(normalized_y * (img_height - 1))

  return [pixel_x,pixel_y,pos[2]]


def antennaVerticalImg(save_path,id):
  PI= np.pi
  phi = np.linspace(-PI, PI, 100)
  theta = np.linspace(0.0, PI, 100)
  #c_theta, c_phi = scene.tx_array.antenna._patterns[0]((PI/2*tf.ones_like(phi)) , tf.constant(phi, tf.float32))
  c_theta, c_phi = scene.tx_array.antenna._patterns[0](theta, np.zeros_like(theta))
  c_theta = c_theta.numpy()
  c_phi = c_phi.numpy()
  g = np.abs(c_theta)*2 + np.abs(c_phi)*2
  g = np.where(g==0, 1e-12, g)
  g_db = 10*np.log10(g)
  g_db_max = np.max(g_db)
  g_db_min = np.min(g_db)
  if g_db_min==g_db_max:
      g_db_min = -30
  else:
      g_db_min = np.maximum(-60., g_db_min)
  theimg = np.tile(g_db,(100, 1))
  plt.axis('off')
  plt.ioff()
  plt.imshow(theimg)

  image_path = os.path.join(save_path, f'Antenna_Vert_{id}.png')

  plt.savefig(image_path,bbox_inches='tight', pad_inches=0)
  print(f"Image saved to {image_path}")
  return

def antennaHorzImg(save_path,id):
  PI= np.pi
  numpoints = 180
  phi = np.linspace(-PI, PI, 360)
  c_theta, c_phi = scene.tx_array.antenna._patterns[0]((PI/2*tf.ones_like(phi)) , tf.constant(phi, tf.float32))
  c_theta = c_theta.numpy()
  c_phi = c_phi.numpy()
  g = np.abs(c_theta)*2 + np.abs(c_phi)*2
  g = np.where(g==0, 1e-12, g)
  g_db = 10*np.log10(g)
  g_db_max = np.max(g_db)
  g_db_min = np.min(g_db)
  if g_db_min==g_db_max:
      g_db_min = -30
  else:
      g_db_min = np.maximum(-60., g_db_min)
  theimg = np.tile(g_db,(360, 1))
  plt.axis('off')
  plt.ioff()
  plt.imshow(theimg)

  image_path = os.path.join(save_path, f'Antenna_Horz_{id}.png')

  plt.savefig(image_path,bbox_inches='tight', pad_inches=0)
  print(f"Image saved to {image_path}")
  return image_path


In [ ]:
#this function is reposnislble for combining them into one coverage map
def combineOneVectorMap(cm):
  if cm._value.shape[0] == 1:
    return
  ntimes = cm._value.shape[0]
  resultant_matrix = cm._value[0,:,:]
  for i in range(1,ntimes+1):
    resultant_matrix = tf.math.abs(resultant_matrix - cm._value[i,:,:])

  new_tf = tf.stack([resultant_matrix]*ntimes,axis=0)
  cm._value = new_tf
  return new_tf

In [ ]:
# min_x = float(scene.center[0] - scene.size[0]/2)
# max_x = float(scene.center[0] + scene.size[0]/2)
# min_y = float(scene.center[1] - scene.size[1]/2)
# max_y = float(scene.center[1] + scene.size[1]/2)
# min_z = float(scene.center[2] - scene.size[2]/2)
# max_z = float(scene.center[2] + scene.size[2]/2)

# poss = scene.transmitters["tx"].position.numpy()
# orrr = scene.transmitters["tx"].orientation.numpy()
# orrr[0] = 0.5
# print(poss)
# print(orrr)
#save_antenna_orientation([poss],[orrr],min_x,max_x,min_y,max_y,[655,500],'./images',1)

In [ ]:
counter = 0
import time
#paramters for each loop
num_freq_point = 1
num_tranmitters = 1
random_trans_loc = 5
num_random_ori = 5
min_freq_val = 2e9
max_freq_val = 6e9
freq_save_paths = './images/frequencies'
os.makedirs(freq_save_paths, exist_ok=True)
os.makedirs('./images/verticalpattern_images', exist_ok=True)
os.makedirs('./images/horzpattern_images', exist_ok=True)
os.makedirs('./images/azimuth', exist_ok=True)
os.makedirs('./images/tilt', exist_ok=True)
os.makedirs('./images/height', exist_ok=True)
os.makedirs('./images/position', exist_ok=True)
os.makedirs('./images/results', exist_ok=True)
random.seed(0) #set the seed into this number so that data can be repeated


min_x = float(scene.center[0] - scene.size[0]/2)
max_x = float(scene.center[0] + scene.size[0]/2)
min_y = float(scene.center[1] - scene.size[1]/2)
max_y = float(scene.center[1] + scene.size[1]/2)
min_z = float(scene.center[2] - scene.size[2]/2)
max_z = float(scene.center[2] + scene.size[2]/2)

min_yaw = 0
max_yaw = 2 *np.pi
min_pitch = -np.pi/2
max_pitch = np.pi/2
min_roll = -np.pi/2
max_roll = np.pi/2

min_height = 0.5
max_height = 500

image_coords_arr = [[0,0,0]]*5

remove_all_transmitters(scene)
whole_loop = time.time()
for freq in range(0,num_freq_point):
  scene.frequency = random.uniform(min_freq_val,max_freq_val)   #getting random frequency
  freq_current_img_path = create_colored_image_from_number(int(scene.frequency),freq_save_paths)   #making the image
  for num_trans in range(0,num_tranmitters):
    scene.add(trans_list[num_trans])
    for num_random_loc in range(0,random_trans_loc):
      #act_num_ant = len(scene.transmitters)
      act_num_ant = num_trans + 1
      for pos_sel in range(0,act_num_ant):
        x_rand = random.uniform(float(scene.center[0] - scene.size[0]/2), float(scene.center[0] + scene.size[0]/2))
        y_rand = random.uniform(float(scene.center[1] - scene.size[1]/2), float(scene.center[1] + scene.size[1]/2))
        z_rand = random.uniform(float(scene.center[2] - scene.size[2]/2), float(scene.center[2] + scene.size[2]/2))
        trans_list[pos_sel].position = [x_rand,y_rand,z_rand]
        print([x_rand,y_rand,z_rand])
        #getting image coords
        image_coords_arr[pos_sel] = sionna_to_image_cord([x_rand,y_rand,z_rand] , min_x,max_x,min_y,max_y,[655,500])

      start_time = time.time()
      for ori_itr in range(0,num_random_ori):
        azi_arr = []
        for ori_det_itr in range(0,act_num_ant):
          yaw_rand = random.uniform(min_yaw, max_yaw)
          pitch_rand = random.uniform(min_pitch,max_pitch)
          roll_rand = random.uniform(min_roll,max_roll)
          trans_list[pos_sel].orientation = [yaw_rand,pitch_rand,roll_rand]

          #antenna_mainobe_dir = euler_to_cartesian(yaw_rand,pitch_rand,roll_rand)
          #azi_val_itr = calculate_azimuth_angles(655,500,image_coords_arr[0][:2],antenna_mainobe_dir[:2])
          #azi_arr.append(azi_val_itr)



        #getting the azimuth images here
        #azimuth_image_path = plot_azimuth(azi_arr,'./images',counter,1)
        vertical_image_path = antennaVerticalImg('./images/verticalpattern_images',counter)
        horz_image_path = antennaHorzImg('./images/horzpattern_images',counter)

        #getting the azimith and the tilt
        #antenna_mainobe_dir = euler_to_cartesian(yaw_rand,pitch_rand,roll_rand)
        azimuth_image_path = number_to_greyscale_image(trans_list[0].orientation.numpy()[0],'./images/azimuth',counter,"azimuth",min_yaw,max_yaw)
        tilt_image_path = number_to_greyscale_image(trans_list[0].orientation.numpy()[2],'./images/tilt',counter,"tilt",min_pitch,max_pitch)

        #getting the tilt image here function name should be changed
        #tilt_val = calculate_tilt(500,655,image_coords_arr[0],antenna_mainobe_dir)
        #tilt_image_path = plot_azimuth([tilt_val],'./images',counter,0)



        #assuming they are one antenna so we can call this function
        #if they are not another ready fucntion exsits
        #height_image_path = number_to_viridis_image(trans_list[ori_det_itr].position[2], './images',counter)
        height_image_path = number_to_greyscale_image(trans_list[0].position[2],'./images/height',counter,"trans_height",min_height,max_height)



        #this paramters need to change a little bit they are simple case
        cm_1 = scene.coverage_map(max_depth=5,
                          diffraction=True, # Disable to see the effects of diffraction
                          cm_cell_size=(5., 5.), # Grid size of coverage map cells in m
                          combining_vec=None,
                          precoding_vec=None,
                          num_samples=int(5e6)) # Reduce if your hardware does not have enough memory
        #combineOneVectorMap(cm_1)
        #getting the transmitter image locations
        active_trans = [trans.position for trans in trans_list][:act_num_ant]
        transpos_image_path = save_antenna_pos(active_trans,min_x,max_x,min_y,max_y,[655,500],'./images/position',counter)

        write_to_excel(workbook, sheet_name, "A" + str(counter + 2), base_map_image_path)    #base map writing
        write_to_excel(workbook, sheet_name, "B" + str(counter + 2), freq_current_img_path) #writing the frequency location
        #write_to_excel(workbook, sheet_name, "C" + str(counter + 2), terrain_image_path) #writing the terrrain
        write_to_excel(workbook, sheet_name, "D" + str(counter + 2), azimuth_image_path) #writing the azimuth
        write_to_excel(workbook, sheet_name, "E" + str(counter + 2), tilt_image_path) #writing the tilt
        write_to_excel(workbook, sheet_name, "F" + str(counter + 2), transpos_image_path) #antenna position
        write_to_excel(workbook, sheet_name, "G" + str(counter + 2), height_image_path) #writing the height
        write_to_excel(workbook, sheet_name, "H" + str(counter + 2), horz_image_path) #writing 2d pattern horizontal
        write_to_excel(workbook, sheet_name, "I" + str(counter + 2), vertical_image_path) #writing 2d pattern vertical

        #write_to_excel(workbook, sheet_name, "D" + str(counter + 2), vertical_pattern_path) #writing both the vertical and horicontal patterns
        #write_to_excel(workbook, sheet_name, "E" + str(counter + 2), horizontal_pattern_path)
        #write_to_excel(workbook, sheet_name, "F" + str(counter + 2), trans_image_path) #antenna position
        end_time = time.time()
        scene.render_to_file(bird_cam,'./images/results/res_'+ str(counter)+".png",coverage_map=cm_1,resolution=(655, 500))
        write_to_excel(workbook, sheet_name, "J" + str(counter + 2), './images/results/res_'+ str(counter)+".png")
        counter = counter + 1
save_excel(workbook, file_name)
whole_end = time.time()
execution_time = end_time - start_time
whole_time = whole_loop -whole_end
print(execution_time)
print(whole_time)


the main missing things currently


*  loop on frequencies
*   make other images like frequency encoder
*   make antenna pattern image saver
*   figure out how the file paths will go
- if you will add cars then you should also control it
-make the antenna arrays 4g and 5g depending on the specfications
-


-
-

-

In [ ]:
#this is for how to download in colab
# !zip -r /content/images_gen.zip /content/images
# from google.colab import files
# files.download("/content/images_gen.zip")